# GENOMIC PREDICTION FOR LONGITUDINAL RESIDUAL FEED INTAKE -

## **COMPUTING CORRECTED PHENOTYPES**






This program is used to computed the corrected phenotypes obtained with different models Random regression, SAD and Multiple-trait models

In [27]:
# Load required packages
library(pacman)
pacman::p_load(dplyr,reshape2, stringr, plyr, psych, corrplot)



# set path work drive
#path=getwd()
setwd("C:/Users/HP/OneDrive/RFI_all/RFI_all")


# Read data 
pheno <- read.table("pheno1.dat", quote="\"", comment.char="")


# renames columns
names(pheno)<-c("animal","identif" ,"bande",
                "sex" ,"lignee"  ,"gener",
                "loge"  , "eleve","pds_dc",  
                "pds_fc" ,  "age" ,  "BW","BWpre",
                "age_dc",
                "temps","FCR","ADG","AMBW"  ,
                "BF","FI" ,"RFI","week","pe1",
                "pe2","pe3",
                "selec1","selec2","e_RR",
                "e_RRss","e_SAD","e_SADss","e_MTA","e_MTH")

pheno <- select(pheno, -c(pe1,pe3)) # drop unnecessary columns


This dataset contains the followings: 
- animal, identif = id of animal
- BW = Body weight
- BWpre = predicted BW using a linear interpolation
- temps = time (in week, from week 4 to week 13 from the beginning of the test)
- pe1 = period 1 (week 1-2)
- pe2 = period 2 (week 3-5)


 **Display the first lines of dataset 'pheno'**

In [28]:
head(pheno)

,animal,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,pds_fc,...,week,pe2,selec1,selec2,e_RR,e_RRss,e_SAD,e_SADss,e_MTA,e_MTH
,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,<int>,...,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,1,1,0,1,-31.580,-30.980,-0.0009618,-9.739e-04,-28.78,-28.560
2,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,2,1,0,1,-4.791,-4.344,-0.0001196,-9.429e-05,-1.39,-1.137
3,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,3,2,0,1,16.910,17.230,0.0002545,2.841e-04,19.38,20.080
4,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,4,2,0,1,16.470,16.690,0.0006243,6.332e-04,18.52,19.210
5,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,5,2,0,1,-60.050,-59.900,-0.0016420,-1.647e-03,-58.44,-57.750
6,17MAG20005072,17MAG20005072,200011,2,n,G0,72,17MAG,24,128,...,6,3,0,1,13.970,14.090,0.0004870,4.940e-04,11.81,11.600


In [29]:
length(unique(pheno$animal))

[1] 2397

In above file contains data of 2397 animals, including batch(bande), sex, line (lignee), loge, eleve, BW at the begining and BW_at the end of test

 # Random regression models with matrix A

In [30]:
# Read data (outputs of Asreml program)
sln_RRAe <- read.table("ide_RRA",quote="\"") # Permanental random effect
sln_RRAg <- read.table("sln_RRA",quote="\"") # Genetic random effect
sln_RRAg$V3=sln_RRAe$V3 + sln_RRAg$V3

In [31]:
sln_RRAg = cbind(sln_RRAg,as.data.frame(matrix(unlist( strsplit(as.character(sln_RRAg$V2)  , "\\.")) , ncol = 2 , byrow = TRUE )))
names(sln_RRAg)<-c("kk","a","para","SE","gr","animal")
head(sln_RRAg)

,kk,a,para,SE,gr,animal
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
1,"leg(temps,2).animal",1.17M3019830268,-0.0459300,8.166,1,17M3019830268
2,"leg(temps,2).animal",1.17M3019830280,0.0009301,8.166,1,17M3019830280
3,"leg(temps,2).animal",1.17M3019830492,-0.0234300,8.166,1,17M3019830492
4,"leg(temps,2).animal",1.17M3019830508,-0.0234300,8.166,1,17M3019830508
5,"leg(temps,2).animal",1.17M3019840010,-0.0215700,8.166,1,17M3019840010
6,"leg(temps,2).animal",1.17M3019840036,-0.0468600,8.166,1,17M3019840036


In [32]:
sln_RRAg$para1=ifelse(sln_RRAg$gr==1,sln_RRAg$para,0)  
sln_RRAg$para2=ifelse(sln_RRAg$gr==2,sln_RRAg$para,0) 
sln_RRAg$para3=ifelse(sln_RRAg$gr==3,sln_RRAg$para,0)  
sln_RRAg=arrange(sln_RRAg,animal,gr)  
head(sln_RRAg)

,kk,a,para,SE,gr,animal,para1,para2,para3
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,"leg(temps,2).animal",1.0000019851067,0.019560,8.166,1,0000019851067,0.01956,0.00000,0.000000
2,"leg(temps,2).animal",2.0000019851067,0.035280,7.498,2,0000019851067,0.00000,0.03528,0.000000
3,"leg(temps,2).animal",3.0000019851067,0.006404,4.539,3,0000019851067,0.00000,0.00000,0.006404
4,"leg(temps,2).animal",1.0000019858909,-0.152500,8.165,1,0000019858909,-0.15250,0.00000,0.000000
5,"leg(temps,2).animal",2.0000019858909,-0.161000,7.497,2,0000019858909,0.00000,-0.16100,0.000000
6,"leg(temps,2).animal",3.0000019858909,-0.025930,4.538,3,0000019858909,0.00000,0.00000,-0.025930


In [33]:
tatemp <- read.table("OP.txt",quote="\"") 
names(tatemp)[1]<-"temps"  
names(tatemp)[2]<-"f1"  
names(tatemp)[3]<-"f2"  
names(tatemp)[4]<-"f3"  
Temp=t(as.matrix(tatemp[2:4]))


In [34]:
# matix T
head(Temp)

f1,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711,0.70711
f2,-1.22474,-0.95258,-0.68041,-0.40825,-0.13608,0.13608,0.40825,0.68041,0.95258,1.22474
f3,1.58114,0.64417,-0.05856,-0.52705,-0.76129,-0.76129,-0.52705,-0.05856,0.64417,1.58114


In [35]:
#sln1=merge(sln,pheno,by = "animal")

SLN=sln_RRAg[,c("animal","gr","para")]
SLN1=arrange(SLN,animal,gr)
#SLN2=reshape(SLN1, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN2=reshape(SLN1,varying = NULL, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN3=arrange(SLN2,animal)




SLN4=SLN3[,2:4]
SLN5=as.matrix(SLN4)
EBV_all=as.data.frame((SLN5)%*%(Temp))
EBV_all$animal=SLN2$animal


df_effect=melt(EBV_all, id.vars = "animal", 
               measure.vars = c("V1", "V2","V3",
                                "V4","V5","V6","V7","V8","V9","V10"))

df_effect = arrange(df_effect,animal,variable)
df_effect$week=c(rep(1:10,3986))

In [36]:
head(df_effect)

,animal,variable,value,week
,<fct>,<fct>,<dbl>,<int>
1,0000019851067,V1,-0.019252135,1
2,0000019851067,V2,-0.015650686,2
3,0000019851067,V3,-0.010548811,3
4,0000019851067,V4,-0.003947217,4
5,0000019851067,V5,0.004154868,5
6,0000019851067,V6,0.013756673,6


In [37]:
pheno1=merge(df_effect,pheno,by=c("animal","week"))
pheno2=arrange(pheno1,animal,week)
pheno2$y_RR = pheno2$e_RR +pheno2$value

pheno2=subset(pheno2,select=-c(variable,value))
write.table(pheno2,"dat_correct_RRA")

In [38]:
head(pheno2)
summary(pheno2)

,animal,week,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,...,pe2,selec1,selec2,e_RR,e_RRss,e_SAD,e_SADss,e_MTA,e_MTH,y_RR
,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,...,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,2,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,1,0,1,74.510,74.850,1.718e-03,1.719e-03,74.170,74.060,94.97610
2,17MAG20005070,4,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,2,0,1,67.060,67.430,1.574e-03,1.593e-03,63.490,64.150,79.52834
3,17MAG20005070,5,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,2,0,1,-18.340,-17.980,-6.276e-04,-6.220e-04,-27.610,-26.970,-11.37910
4,17MAG20005070,6,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,3,0,1,-29.070,-28.730,-4.628e-04,-4.519e-04,-17.430,-17.220,-28.62189
5,17MAG20005070,7,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,3,0,1,-6.171,-5.867,1.343e-05,2.321e-05,-2.308,-2.115,-13.24151
6,17MAG20005070,8,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,3,0,1,-8.770,-8.502,-1.026e-04,-1.043e-04,-13.520,-13.330,-24.36453


           animal           week                 identif          bande       
 17MAG20005072:   10   Min.   : 1.000   17MAG20005072:   10   Min.   :200011  
 17MAG20005146:   10   1st Qu.: 3.000   17MAG20005146:   10   1st Qu.:200406  
 17MAG20005197:   10   Median : 5.000   17MAG20005197:   10   Median :200612  
 17MAG20005251:   10   Mean   : 5.468   17MAG20005251:   10   Mean   :200613  
 17MAG20005297:   10   3rd Qu.: 8.000   17MAG20005297:   10   3rd Qu.:200812  
 17MAG20005301:   10   Max.   :10.000   17MAG20005301:   10   Max.   :201111  
 (Other)      :22893                    (Other)      :22893                   
      sex        lignee        gener           loge          eleve      
 Min.   :1.000   -:11744   G7     :4970   Min.   : 71.00   17MAG:13355  
 1st Qu.:2.000   +:10456   G6     :3748   1st Qu.: 81.00   86001: 9598  
 Median :2.000   n:  753   G5     :3218   Median : 91.00                
 Mean   :2.016             G4     :3107   Mean   : 87.89                
 3r

## Random regression models with matrix H

In [39]:
# Random regression model with matrix H
#rm(list = ls()) # ecrase previous memories
###############################################################################################
pheno2=read.table("dat_correct_RRA")


sln_RRHe <- read.table("ide_RRH",quote="\"") # Permanental random effect
sln_RRHg <- read.table("sln_RRH",quote="\"") # Genetic random effect
sln_RRHg$V3=sln_RRHe$V3 + sln_RRHg$V3

# sln <- read.table("C:/Users/vhtran/Google Drive/these/RFI/INRA/validation/ide_RRA",quote="\"") # 
# sln_1 <- read.table("C:/Users/vhtran/Google Drive/these/RFI/INRA/validation/sln_RRA",quote="\"") 
m_RRH <- as.data.frame(matrix( unlist( strsplit(as.character(sln_RRHg$V2)  , "\\." ) ) , ncol = 2 , byrow = TRUE ))
sln_RRHg=cbind(sln_RRHg,m_RRH)
names(sln_RRHg)<-c("kk","a","para","SE","gr","animal")



In [40]:
head(sln_RRHg)

,kk,a,para,SE,gr,animal
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
1,"leg(temps,2).giv(ani",1.17M3019830268,-0.11520,8.057,1,17M3019830268
2,"leg(temps,2).giv(ani",1.17M3019830280,-0.00814,8.057,1,17M3019830280
3,"leg(temps,2).giv(ani",1.17M3019830492,-0.05354,8.057,1,17M3019830492
4,"leg(temps,2).giv(ani",1.17M3019830508,-0.05354,8.057,1,17M3019830508
5,"leg(temps,2).giv(ani",1.17M3019840010,-0.06982,8.057,1,17M3019840010
6,"leg(temps,2).giv(ani",1.17M3019840036,-0.10710,8.057,1,17M3019840036


In [41]:
sln_RRHg$para1=ifelse(sln_RRHg$gr==1,sln_RRHg$para,0)  
sln_RRHg$para2=ifelse(sln_RRHg$gr==2,sln_RRHg$para,0) 
sln_RRHg$para3=ifelse(sln_RRHg$gr==3,sln_RRHg$para,0)  
sln_RRHg=arrange(sln_RRHg,animal,gr)  
head(sln_RRHg)


,kk,a,para,SE,gr,animal,para1,para2,para3
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,"leg(temps,2).giv(ani",1.0000019851067,0.018410,8.057,1,0000019851067,0.01841,0.00000,0.000000
2,"leg(temps,2).giv(ani",2.0000019851067,0.031990,7.319,2,0000019851067,0.00000,0.03199,0.000000
3,"leg(temps,2).giv(ani",3.0000019851067,0.004749,3.771,3,0000019851067,0.00000,0.00000,0.004749
4,"leg(temps,2).giv(ani",1.0000019858909,-0.243100,8.059,1,0000019858909,-0.24310,0.00000,0.000000
5,"leg(temps,2).giv(ani",2.0000019858909,-0.397000,7.320,2,0000019858909,0.00000,-0.39700,0.000000
6,"leg(temps,2).giv(ani",3.0000019858909,0.016640,3.771,3,0000019858909,0.00000,0.00000,0.016640


In [42]:
SLN_RRH=sln_RRHg[,c("animal","gr","para")]
SLN_RRH1=arrange(SLN_RRH,animal,gr)
#SLN2=reshape(SLN1, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN_RRH2=reshape(SLN_RRH1,varying = NULL, timevar = "gr",idvar = "animal",direction="wide",sep="")
SLN_RRH3=arrange(SLN_RRH2,animal)



SLN_RRH4=SLN_RRH3[,2:4]
SLN_RRH5=as.matrix(SLN_RRH4)
EBV_all1=as.data.frame((SLN_RRH5)%*%(Temp))
EBV_all1$animal=SLN_RRH2$animal

In [43]:
head(EBV_all1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,animal
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-0.01865270,-0.01439598,-0.009026522,-0.002544983,0.00504933,0.01375573,0.02357485,0.03450611,0.04655009,0.05970616,0000019851067
2,0.34063351,0.21699481,0.097249891,-0.018593303,-0.13054255,-0.23859007,-0.34274380,-0.44299565,-0.53935371,-0.63181005,0000019858909
3,-0.01865270,-0.01439598,-0.009026522,-0.002544983,0.00504933,0.01375573,0.02357485,0.03450611,0.04655009,0.05970616,0000019858971
4,0.05959042,0.03349862,0.010659931,-0.008924683,-0.02525605,-0.03833333,-0.04815751,-0.05472747,-0.05804432,-0.05810709,0000019859155
5,0.26493798,0.21916376,0.175319683,0.133408547,0.09342764,0.05537967,0.01926185,-0.01492295,-0.04717761,-0.07749932,0000019860769
6,0.26493798,0.21916376,0.175319683,0.133408547,0.09342764,0.05537967,0.01926185,-0.01492295,-0.04717761,-0.07749932,0000019860909


In [44]:
df_effect1=melt(EBV_all1, 
               id.vars = "animal", 
               measure.vars = c("V1", "V2","V3","V4","V5","V6","V7","V8","V9","V10"))

df_effect1= arrange(df_effect1,animal,variable)
df_effect1$week=c(rep(1:10,3986))

# combine the last dataset pheno2 with EBV from RR with matrix H
pheno3=merge(df_effect1,pheno2,by=c("animal","week"))
pheno4=arrange(pheno3,animal,week)
pheno4$y_RRH = pheno4$e_RRss+pheno4$value
pheno4=subset(pheno4,select=-c(variable,value))

#write.table(pheno4,"dat_correct_RRA_RRH")

In [45]:
head(pheno4)

,animal,week,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,...,selec1,selec2,e_RR,e_RRss,e_SAD,e_SADss,e_MTA,e_MTH,y_RR,y_RRH
,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,...,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,2,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,74.510,74.850,1.718e-03,1.719e-03,74.170,74.060,94.97610,95.87767
2,17MAG20005070,4,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,67.060,67.430,1.574e-03,1.593e-03,63.490,64.150,79.52834,79.07920
3,17MAG20005070,5,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,-18.340,-17.980,-6.276e-04,-6.220e-04,-27.610,-26.970,-11.37910,-12.41159
4,17MAG20005070,6,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,-29.070,-28.730,-4.628e-04,-4.519e-04,-17.430,-17.220,-28.62189,-30.16971
5,17MAG20005070,7,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,-6.171,-5.867,1.343e-05,2.321e-05,-2.308,-2.115,-13.24151,-15.24268
6,17MAG20005070,8,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,0,1,-8.770,-8.502,-1.026e-04,-1.043e-04,-13.520,-13.330,-24.36453,-26.74102


## SAD models with matrix A

In [53]:
###############################################################################################
pheno4=read.table("dat_correct_RRA_RRH")



sln_sad <- read.table("sln_sadA", quote="\"") 
ide_sad<- read.table("ide_sadA", quote="\"")
sln_sad$ide_sad = ide_sad$V3


sln_sad$somme= sln_sad$V3 + sln_sad$ide_sad
names(sln_sad)<-c("V1","code","g","SEg","e","somme")



m_sadA <- as.data.frame(matrix( unlist( strsplit(as.character(sln_sad$code)  , "\\." ) ) , ncol = 2 , byrow = TRUE ))

sln_sad1=cbind(sln_sad,m_sadA)
names(sln_sad1)<-c("V1","code","g","SEgA","e","somme","temps","animal")
sln_sad1$temps=as.numeric(levels(sln_sad1$temps))[sln_sad1$temps]
sln_sad2=arrange(sln_sad1,animal,temps)
N_animal=length(unique(sln_sad2$animal))  #number animal from SAD.sln



sln_sad2$week=sln_sad2$temps
sln_sad2=subset(sln_sad2,select=-c(temps))
critere=c("animal","week")
DF_s=merge(pheno4,sln_sad2,by=c(critere))
DF_s$y_SAD = DF_s$e_SAD + DF_s$somme

DF_s <- select(DF_s, -c(somme,code, V1,g,e))

head(DF_s)
write.table(DF_s,"dat_correct_RRA_RRH_sadA")
############################################################


,animal,week,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,...,e_RR,e_RRss,e_SAD,e_SADss,e_MTA,e_MTH,y_RR,y_RRH,SEgA,y_SAD
,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,10,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-1.120,-0.9432,-2.249e-04,-2.081e-04,-5.578,-5.368,-36.77958,-39.69193,11.780,-36.55022
2,17MAG20005070,2,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,74.510,74.8500,1.718e-03,1.719e-03,74.170,74.060,94.97610,95.87767,8.846,92.79172
3,17MAG20005070,4,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,67.060,67.4300,1.574e-03,1.593e-03,63.490,64.150,79.52834,79.07920,7.179,78.04357
4,17MAG20005070,5,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-18.340,-17.9800,-6.276e-04,-6.220e-04,-27.610,-26.970,-11.37910,-12.41159,7.112,-11.63063
5,17MAG20005070,6,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-29.070,-28.7300,-4.628e-04,-4.519e-04,-17.430,-17.220,-28.62189,-30.16971,7.418,-29.23046
6,17MAG20005070,7,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-6.171,-5.8670,1.343e-05,2.321e-05,-2.308,-2.115,-13.24151,-15.24268,8.000,-14.00499


In [50]:
fix(DF_s)

In [54]:
###################################################
#rm(list = ls())


###############################################################################################


DF_s=read.table("dat_correct_RRA_RRH_sadA")

sln_sadH <- read.table("sln_sadH", quote="\"") #old is G_sad
ide_sadH <- read.table("ide_sadH", quote="\"")
sln_sadH$ide_sad = ide_sadH$V3

sln_sadH$somme= sln_sadH$V3 + sln_sadH$ide_sad
names(sln_sadH)<-c("V1","code","g","SEg","e","somme")

m_sadH <- as.data.frame(matrix(unlist( strsplit(as.character(sln_sadH$code)  , "\\." )) , ncol = 2 , byrow = TRUE ))


sln_sadH1=cbind(sln_sad,m_sadH)
names(sln_sadH1)<-c("V11","code1","g1","SEgH","e1","somme1","temps","animal")

sln_sadH1$temps=as.numeric(levels(sln_sadH1$temps))[sln_sadH1$temps]
sln_sadH2=arrange(sln_sadH1,animal,temps)
N_animal=length(unique(sln_sadH2$animal))  #number animal from SAD.sln
sln_sadH2$week=sln_sadH2$temps
sln_sadH2=select(sln_sadH2,-c(temps))




critere=c("animal","week")
DF_s2 =merge(DF_s,sln_sadH2,by=c(critere))


DF_s2$y_SADH = DF_s2$e_SADss + DF_s2$somme1

# Delete some columns
DF_s2=select(DF_s2,-c(code1,somme1,V11,code1,g1,e1))
# write table
write.table(DF_s2,"dat_correct_RRA_RRH_sadA_SADH")

In [55]:
head(DF_s2)

,animal,week,identif,bande,sex,lignee,gener,loge,eleve,pds_dc,...,e_SAD,e_SADss,e_MTA,e_MTH,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH
,<fct>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,10,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-2.249e-04,-2.081e-04,-5.578,-5.368,-36.77958,-39.69193,11.780,-36.55022,11.780,-36.55021
2,17MAG20005070,2,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,1.718e-03,1.719e-03,74.170,74.060,94.97610,95.87767,8.846,92.79172,8.846,92.79172
3,17MAG20005070,4,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,1.574e-03,1.593e-03,63.490,64.150,79.52834,79.07920,7.179,78.04357,7.179,78.04359
4,17MAG20005070,5,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-6.276e-04,-6.220e-04,-27.610,-26.970,-11.37910,-12.41159,7.112,-11.63063,7.112,-11.63062
5,17MAG20005070,6,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,-4.628e-04,-4.519e-04,-17.430,-17.220,-28.62189,-30.16971,7.418,-29.23046,7.418,-29.23045
6,17MAG20005070,7,17MAG20005070,200011,2,n,G0,71,17MAG,25,...,1.343e-05,2.321e-05,-2.308,-2.115,-13.24151,-15.24268,8.000,-14.00499,8.000,-14.00498


## Multiple trait models

In [62]:
# rm(list = ls())

# Read files
pheno3=read.table("dat_correct_RRA_RRH_sadA_SADH")
sln_mt <- read.table("sln_mtA", quote="\"", comment.char="") #old is G_sad
ide_mt <- read.table("ide_mtA", quote="\"", comment.char="")
sln_mt$V3=sln_mt$V3+ide_mt$V3
names(sln_mt)<-c("V1","code","value","SE")





m_mtA <- as.data.frame(matrix( unlist(strsplit(as.character(sln_mt$code)  , "\\.")) , ncol = 2 , byrow = TRUE ))
sln_mt1=cbind(sln_mt,m_mtA)
names(sln_mt1)<-c("V1","code","value","SEg_MTA","pe2","animal")
sln_mt1$pe2=as.numeric(levels(sln_mt1$pe2))[sln_mt1$pe2]
sln_mt2=arrange(sln_mt1,animal,pe2)
N_animal=length(unique(sln_mt2$animal))  #number animal from SAD.sln


critere=c("animal","pe2")
DF_s3=merge(DF_s2,sln_mt2,by=c(critere))
DF_s3$y_MT = DF_s3$e_MTA+DF_s3$value # corrected phenotypes
DF_s3 = select(DF_s3,-c(value, code, V1)) # drop some columns
write.table(DF_s3,"dat_correct_RRA_RRH_sadA_SADH_mtA")
########################################################################################################

head(DF_s3)


,animal,pe2,week,identif,bande,sex,lignee,gener,loge,eleve,...,e_MTA,e_MTH,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH,SEg_MTA,y_MT
,<fct>,<int>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,1,2,17MAG20005070,200011,2,n,G0,71,17MAG,...,74.170,74.060,94.97610,95.87767,8.846,92.79172,8.846,92.79172,7.612,95.4500
2,17MAG20005070,2,4,17MAG20005070,200011,2,n,G0,71,17MAG,...,63.490,64.150,79.52834,79.07920,7.179,78.04357,7.179,78.04359,5.936,80.2270
3,17MAG20005070,2,5,17MAG20005070,200011,2,n,G0,71,17MAG,...,-27.610,-26.970,-11.37910,-12.41159,7.112,-11.63063,7.112,-11.63062,5.936,-10.8730
4,17MAG20005070,3,6,17MAG20005070,200011,2,n,G0,71,17MAG,...,-17.430,-17.220,-28.62189,-30.16971,7.418,-29.23046,7.418,-29.23045,6.505,-28.2649
5,17MAG20005070,3,7,17MAG20005070,200011,2,n,G0,71,17MAG,...,-2.308,-2.115,-13.24151,-15.24268,8.000,-14.00499,8.000,-14.00498,6.505,-13.1429
6,17MAG20005070,3,8,17MAG20005070,200011,2,n,G0,71,17MAG,...,-13.520,-13.330,-24.36453,-26.74102,8.805,-25.16910,8.805,-25.16910,6.505,-24.3549


In [70]:
########################################################################################################
#rm(list = ls())



DF_s3=read.table("dat_correct_RRA_RRH_sadA_SADH_mtA")
sln_mtH <- read.table("sln_mtH", quote="\"", comment.char="") #old is G_sad
ide_mtH <- read.table("ide_mtH", quote="\"", comment.char="")
sln_mtH$V3=sln_mtH$V3+ide_mtH$V3
names(sln_mtH)<-c("V1","code","value","SE")




m_mtH <- as.data.frame(matrix(unlist(strsplit(as.character(sln_mtH$code)  , "\\." )), ncol = 2 , byrow = TRUE ))
sln_mtH1=cbind(sln_mtH,m_mtH)
names(sln_mtH1)<-c("V11","code1","value1","SEg_MTH","pe2","animal")
head(sln_mtH1)
sln_mtH1 <- select(sln_mtH1,c(animal,pe2,value1,SEg_MTH))

,V11,code1,value1,SEg_MTH,pe2,animal
,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
1,"period1.giv(animal,1",1.17M3019830268,0.10790,7.714,1,17M3019830268
2,"period1.giv(animal,1",1.17M3019830280,0.02548,7.714,1,17M3019830280
3,"period1.giv(animal,1",1.17M3019830492,0.04120,7.714,1,17M3019830492
4,"period1.giv(animal,1",1.17M3019830508,0.04120,7.714,1,17M3019830508
5,"period1.giv(animal,1",1.17M3019840010,0.09216,7.714,1,17M3019840010
6,"period1.giv(animal,1",1.17M3019840036,0.08239,7.714,1,17M3019840036


In [72]:
critere=c("animal","pe2")
DF_s4=merge(DF_s3,sln_mtH1,by=c(critere))
DF_s4$y_MTH = DF_s4$e_MTH+DF_s4$value1

DF_s4=subset(DF_s4,select=-c(value1))
write.table(DF_s4,"dat_correct_RRA_RRH_sadA_SADH_mtA_mtH")


with(DF_s4,cor(y_RR,y_SADH))
with(DF_s4,cor(y_RRH,y_RR))
head(DF_s4)


[1] 0.9983941

[1] 0.9960819

,animal,pe2,week,identif,bande,sex,lignee,gener,loge,eleve,...,y_RR,y_RRH,SEgA,y_SAD,SEgH,y_SADH,SEg_MTA,y_MT,SEg_MTH,y_MTH
,<fct>,<int>,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<fct>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,17MAG20005070,1,2,17MAG20005070,200011,2,n,G0,71,17MAG,...,94.97610,95.87767,8.846,92.79172,8.846,92.79172,7.612,95.4500,6.915,96.640
2,17MAG20005070,2,4,17MAG20005070,200011,2,n,G0,71,17MAG,...,79.52834,79.07920,7.179,78.04357,7.179,78.04359,5.936,80.2270,5.822,79.649
3,17MAG20005070,2,5,17MAG20005070,200011,2,n,G0,71,17MAG,...,-11.37910,-12.41159,7.112,-11.63063,7.112,-11.63062,5.936,-10.8730,5.822,-11.471
4,17MAG20005070,3,6,17MAG20005070,200011,2,n,G0,71,17MAG,...,-28.62189,-30.16971,7.418,-29.23046,7.418,-29.23045,6.505,-28.2649,6.555,-30.280
5,17MAG20005070,3,7,17MAG20005070,200011,2,n,G0,71,17MAG,...,-13.24151,-15.24268,8.000,-14.00499,8.000,-14.00498,6.505,-13.1429,6.555,-15.175
6,17MAG20005070,3,8,17MAG20005070,200011,2,n,G0,71,17MAG,...,-24.36453,-26.74102,8.805,-25.16910,8.805,-25.16910,6.505,-24.3549,6.555,-26.390


In [73]:
fix(DF_s4)

In [26]:
sessionInfo()


R version 3.6.3 (2020-02-29)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 18363)

Matrix products: default

locale:
[1] LC_COLLATE=English_United States.1252 
[2] LC_CTYPE=English_United States.1252   
[3] LC_MONETARY=English_United States.1252
[4] LC_NUMERIC=C                          
[5] LC_TIME=English_United States.1252    

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] corrplot_0.84   psych_1.9.12.31 plyr_1.8.5      stringr_1.4.0  
[5] reshape2_1.4.3  dplyr_0.8.4     pacman_0.5.1   

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.4       magrittr_1.5     mnormt_1.5-6     tidyselect_1.0.0
 [5] uuid_0.1-2       lattice_0.20-38  R6_2.4.1         rlang_0.4.5     
 [9] tools_3.6.3      grid_3.6.3       parallel_3.6.3   nlme_3.1-144    
[13] htmltools_0.4.0  digest_0.6.25    assertthat_0.2.1 tibble_2.1.3    
[17] crayon_1.3.4     IRdisplay_0.7.0  purrr_0.3.3    

In [74]:
getwd()

[1] "C:/Users/HP/OneDrive/RFI_all/RFI_all"